<a href="https://colab.research.google.com/github/sunhojjang/ai-security/blob/master/Submit_the_1st_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline


Generating Names with a Character-Level RNN
*******************************************


Preparing the Data
==================

.. Note::
   Download the data from
   `here <https://download.pytorch.org/tutorial/data.zip>`_
   and extract it to the current directory.

See the last tutorial for more detail of this process. In short, there
are a bunch of plain text files ``data/names/[Language].txt`` with a
name per line. We split lines into an array, convert Unicode to ASCII,
and end up with a dictionary ``{language: [names ...]}``.




In [0]:
from __future__ import unicode_literals, print_function, division #__future__라는 모듈에서 unicode_literals,print_fuction,division 데이터를 가져온다
from io import open #io라는 모듈에서 새로운 파일을 가져온다
import glob #glob 모듈 전체를 가져온다.
import os #os 모듈 전체를 가져온다.
import unicodedata #unicodedata 모듈 전체를 가져온다.
import string #string 모듈 전체를 가져온다.

all_letters = string.ascii_letters + " .,;'-" #all_letters에 string 모듈의 아스키 코드를 적용
n_letters = len(all_letters) + 1 # n_letters는 all_letters의 크기를 나타냄

#glob함수의 함수값을 반환하는 함수
def findFiles(path): return glob.glob(path)

# 유니코드 문자열을 아스키 코드로 변환하는 함수
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# 파일을 읽고 라인으로 분리하는 함수
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

category_lines = {} #딕셔너리 category_lines 생성
all_categories = [] #리스트 all_categories 생성
for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines #카테고리의 크기만큼 category_lines에 lines를 저장

n_categories = len(all_categories) #n_categories는 all_categories의 크기를 나타냄

if n_categories == 0: #n_categories가 0이라면 예외를 발생시킨다
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

Creating the Network
====================

This network extends `the last tutorial's RNN <#Creating-the-Network>`__
with an extra argument for the category tensor, which is concatenated
along with the others. The category tensor is a one-hot vector just like
the letter input.

We will interpret the output as the probability of the next letter. When
sampling, the most likely output letter is used as the next input
letter.

I added a second linear layer ``o2o`` (after combining hidden and
output) to give it more muscle to work with. There's also a dropout
layer, which `randomly zeros parts of its
input <https://arxiv.org/abs/1207.0580>`__ with a given probability
(here 0.1) and is usually used to fuzz inputs to prevent overfitting.
Here we're using it towards the end of the network to purposely add some
chaos and increase sampling variety.

.. figure:: https://i.imgur.com/jzVrf7f.png
   :alt:





In [0]:
import torch #torch 모듈 가져오기
import torch.nn as nn #torch.nn 모듈을 가져오면서 이름을 nn으로 지정

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size) #샘플링 할때 가장 확률이 높은 문자가 다음 input이 된다
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size) #hidden과 출력값을 결합
        self.o2o = nn.Linear(hidden_size + output_size, output_size) #두 번째 선형레이어를 추가
        self.dropout = nn.Dropout(0.1) #주어진 확률(0.1)로 입력을 무작위로 만든다
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1) #출력값에 hidden값을 결합
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden #출력값과 hidden값을 반환

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

Training
=========
Preparing for Training
----------------------

First of all, helper functions to get random pairs of (category, line):




In [0]:
import random

#카테고리와 줄의 쌍을 얻으려는 함수

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)] # 리스트의 원소를 무작위로 반환

def randomTrainingPair():
    category = randomChoice(all_categories) # 임의의 카레고리를 얻은 후
    line = randomChoice(category_lines[category]) #그 카테고리에서 임의의 줄을 얻기
    return category, line

In [0]:
# 카테고리의 원-핫 벡터
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# 입력을 위해 처음 문자부터 마지막 문자까지의 원-핫 행렬
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# 타겟을 위해 두 번째 문자부터 마지막 문자까지의 LongTensor
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

For convenience during training we'll make a ``randomTrainingExample``
function that fetches a random (category, line) pair and turns them into
the required (category, input, target) tensors.




In [0]:
# 임의의 카테고리, 라인쌍에서 category, input, and target tensors를 만든다
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

Training the Network
--------------------

In contrast to classification, where only the last output is used, we
are making a prediction at every step, so we are calculating loss at
every step.

The magic of autograd allows you to simply sum these losses at each step
and call backward at the end.




In [0]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l #손실들을 합한다

    loss.backward() #모든 단계에서 손실을 계산해야함

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item() / input_line_tensor.size(0)

To keep track of how long training takes I am adding a
``timeSince(timestamp)`` function which returns a human readable string:




In [0]:
import time #time 모듈을 가져온다
import math #math 모듈을 가져온다

#학습에 걸리는 시간을 확인하려고
#문자열을 반환하는 함수
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

Training is business as usual - call train a bunch of times and wait a
few minutes, printing the current time and loss every ``print_every``
examples, and keeping store of an average loss per ``plot_every`` examples
in ``all_losses`` for plotting later.




In [0]:
rnn = RNN(n_letters, 128, n_letters)

n_iters = 100000
print_every = 5000
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time() #현재 시간과 손실을 출력

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every) #평균 손실을 저장
        total_loss = 0

Plotting the Losses
-------------------

Plotting the historical loss from all\_losses shows the network
learning:




In [0]:
import matplotlib.pyplot as plt #matplotlib.pyplot 모듈을 가져오면서 이름을 plt로 지정
import matplotlib.ticker as ticker #matplotlib.ticker모듈을 가져오면서 이름을 ticker로 지정

plt.figure()
plt.plot(all_losses) #손실의 도식화를 통해 네트워크 학습을 보여줌